In [1]:
import ee
ee.Authenticate()
ee.Initialize()
import geemap
ee.Initialize()
import sys
import rioxarray
import geopandas as gpd
sys.path.append(r"C:\Users\JoaoPereira\Desktop\pygeo\Geospatial-Ops")
import aux_functions as af
import importlib
importlib.reload(af)

<module 'aux_functions' from 'c:\\Users\\JoaoPereira\\Desktop\\pygeo\\Geospatial-Ops\\aux_functions.py'>

# Search by WKT

In [2]:
# Your rectangle WKT string, use https://geojson.io/ and save as WKT (open in a text editor to copy the coordinates)

#WKT
wkt_string = "POLYGON ((-9.691880464871304 42.2103384624466, -9.691880464871304 36.708029231300756, -6.1259015927917915 36.708029231300756, -6.1259015927917915 42.2103384624466, -9.691880464871304 42.2103384624466))"

# Manual conversion from WKT to a list of coordinates for Earth Engine
coordinates_str = wkt_string.strip('POLYGON ((').strip('))').split(', ')
coordinates = [[float(coord) for coord in point.split(' ')] for point in coordinates_str]

ee_geometry = ee.Geometry.Polygon([coordinates])

location = ee_geometry.coordinates().getInfo()[::-1]
location

[[[-9.549958547993015, 42.28532298469514],
  [-9.644998817563106, 39.54941956811506],
  [-9.549958547993015, 36.86237642690354],
  [-6.096914983741101, 36.86237642690354],
  [-6.096914983741101, 42.28532298469514],
  [-9.549958547993015, 42.28532298469514]]]

# Dataset Load

In [3]:
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic()

# Mask values below 0 meters
dem_land = dem.updateMask(dem.gte(0))

band_names = dem_land.bandNames().getInfo()
print('Band names:', band_names)

Band names: ['b1']


## Statistics

In [4]:
# Assuming there's only one band, typically this is the case with DEMs
band_name = band_names[0]

# Find minimum and maximum values
stats = dem_land.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee_geometry,
    scale = 30,
    maxPixels=1e13
)

# Print the stats dictionary to inspect the keys
print(stats.getInfo())

# Fetch the results using the correct key names
min_val_key = f'{band_name}_min'
max_val_key = f'{band_name}_max'

min_val = stats.getInfo()[min_val_key]
max_val = stats.getInfo()[max_val_key]

print('Min value:', min_val)
print('Max value:', max_val)

{'b1_max': 2116.7900390625, 'b1_min': 0}
Min value: 0
Max value: 2116.7900390625


In [ ]:
# Parameters for exporting using geemap
scale = 30  # Resolution in meters
filename = r"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation\PT_fabdem.tif" 
# Set export region from WKT
region = ee_geometry  

In [ ]:
## Export DEM locally

In [6]:
def split_region(geometry, n):
    tiles = []
    bounds = geometry.bounds().getInfo()["coordinates"][0]
    x_min, y_min = bounds[0]
    x_max, y_max = bounds[2]
    width = x_max - x_min
    height = y_max - y_min
    tile_width = width / n
    tile_height = height / n
    for i in range(n):
        for j in range(n):
            x1 = x_min + i * tile_width
            y1 = y_min + j * tile_height
            x2 = x1 + tile_width
            y2 = y1 + tile_height
            tiles.append(ee.Geometry.Rectangle([x1, y1, x2, y2]))
    return tiles


In [7]:
# Split the region into 4x4 tiles

n_tiles = 6
tiles = split_region(ee_geometry, n_tiles)

Export task started with task ID: RUYPTVXILEYBPLMV7QPTDE7M
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task still running...
Task completed


In [ ]:
for idx, tile in enumerate(tiles):
    filename = fr"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation\PT_fabdem_tile_{idx}.tif" 
    geemap.ee_export_image(
        dem_land,
        filename=filename,
        scale=30,
        region=tile,
        file_per_band=False,
        crs="EPSG:32629"
    )
    print(f'Tile {idx} has been successfully exported as {filename}')

In [ ]:
af.merge_tiffs(input_dir=r"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation",
               output_tif = fr"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation\PT_fabdem_merged.tif")

## Tile Deletion

In [ ]:
input_dir = r"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation"

input_rasters = [glob.glob(os.path.join(input_dir, f'*_tile_*.{ext}')) for ext in ['tif']]

# Flatten the list of lists into a single list
input_rasters = sum(input_rasters, [])

for raster in input_rasters:
    os.remove(raster)
    print(f"Deleted {raster}")

## Clipping

In [ ]:
data = rioxarray.open_rasterio(r"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation\PT_fabdem_merged.tif")

bounds = gpd.read_file(r"D:\Geospatial_Pessoal\Portugal_Boundaries\pt_dissolved_geojson_3763.geojson")

In [ ]:
if bounds.geometry.type.isin(['MultiPolygon']).any():
    bounds = bounds.explode(index_parts=False)
    print("There are multipolygons in this file and they have been exploded into individual polygons.")
else:
    print("No multipolygons found in this file.")

clipped_fabdem = data.rio.clip(bounds.geometry.tolist(), bounds.crs)

clipped_fabdem.rio.to_raster(r"D:\Geospatial_Pessoal\FFSM\Fatores_Condicionantes\Elevation\PT_fabdem_merged_clipped.tif")